Train a neural network to predict news topic.

![](https://datanizing.com/img/img-cloud1-fs8.png)

[Image Source](https://datanizing.com/workshop/2015/11/29/building-a-tagcloud.html)

Example adapted from: https://github.com/fchollet/keras/blob/master/examples/reuters_mlp.py

In [32]:
reset -fs

In [33]:
from keras.datasets import reuters

`reuters` is a dataset of 11,228 newswires from [Reuters](https://www.reuters.com/)

> National Health Enhancement Systems Inc said it is offering a new health evaluation system to its line of fitness assessment programs. The company said the program, called The Health Test, will be available in 60 days.
> 
> Customers who use the program will receive a computer-generated report and recommendations for implementing a program to improve their physical condition.

Each labeled with 1 of 46 topics. 

For example:

- Business
- World
- Politics
- Tech
- Breakingviews

As with the IMDB dataset, each wire is encoded as a sequence of word indexes (same conventions).

In [34]:
print('Loading data...')
num_words = 1_000
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=1_000,
                                                        skip_top=25,
                                                        test_split=.2)
print('Data loaded.')

print(f'{len(x_train):,} train sequences')
print(f'{len(x_test):,} test sequences')

Loading data...
Data loaded.
8,982 train sequences
2,246 test sequences


In [35]:
# Assume y takes values 0...K-1 where K is number of classes; Add 1 allow for 1 hot encoding
num_classes = max(y_train) + 1 
print(num_classes, 'classes')

46 classes


In [36]:
from keras.preprocessing.text import Tokenizer

In [37]:
print('Vectorizing sequence data...')
tokenizer = Tokenizer(num_words=1_000)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print('Data vectorized.')

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Vectorizing sequence data...
Data vectorized.
x_train shape: (8982, 1000)
x_test shape: (2246, 1000)


In [38]:
from keras import utils

In [39]:
print('Convert class vector to binary class matrix (for use with categorical_crossentropy)')
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (8982, 46)
y_test shape: (2246, 46)


In [40]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

In [41]:
print('Building model...')
model = Sequential()
model.add(Dense(512, input_shape=(1_000,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
print('Model built')

Building model...
Model built


In [42]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 512)               512512    
_________________________________________________________________
activation_5 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 46)                23598     
_________________________________________________________________
activation_6 (Activation)    (None, 46)                0         
Total params: 536,110.0
Trainable params: 536,110.0
Non-trainable params: 0.0
_________________________________________________________________
None


In [43]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

![](https://cdn-images-1.medium.com/max/1600/1*lKc-R1JFGKVjBkX_wmDQfg.jpeg)

In [44]:
history = model.fit(x_train, y_train,
                    batch_size=32,
                    epochs=5,
                    verbose=True,
                    validation_split=0.1)

Train on 8083 samples, validate on 899 samples
Epoch 1/5
8083/8083 [==============================] - 5s - loss: 1.4602 - acc: 0.6745 - val_loss: 1.1083 - val_acc: 0.7664
Epoch 2/5
8083/8083 [==============================] - 5s - loss: 0.7888 - acc: 0.8178 - val_loss: 0.9450 - val_acc: 0.7920
Epoch 3/5
8083/8083 [==============================] - 5s - loss: 0.5374 - acc: 0.8708 - val_loss: 0.8948 - val_acc: 0.8065
Epoch 4/5
8083/8083 [==============================] - 6s - loss: 0.4006 - acc: 0.9037 - val_loss: 0.8974 - val_acc: 0.8065
Epoch 5/5
8083/8083 [==============================] - 7s - loss: 0.3161 - acc: 0.9196 - val_loss: 0.9289 - val_acc: 0.7998


In [45]:
score = model.evaluate(x_test, 
                       y_test,
                       batch_size=32, 
                       verbose=True)

2208/2246 [============================>.] - ETA: 0s

In [46]:
print(f'Test accuracy: {score[1]:.2%}')

Test accuracy: 79.56%


<br>
<br> 
<br>

----